In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scienceplots
import sympy as sp
import pandas as pd
import os
from IPython.core.display import Latex
from sympy import *
plt.style.use(['science','notebook','grid'])
def lprint(*args,**kwargs):
    """Pretty print arguments as LaTeX using IPython display system 
    
    Parameters
    ----------
    args : tuple 
        What to print (in LaTeX math mode)
    kwargs : dict 
        optional keywords to pass to `display` 
    """
    display(Latex('$$'+' '.join(args)+'$$'),**kwargs)

# Importing Example Data for Pendulum and Defining Periods etc.

In [2]:
data_Alice = pd.read_csv('Example Data/data_Alice_pendulum14m_25measurements.dat.txt', delim_whitespace=True, header=None)
data_Alice.columns = ['Period Number', 'Time']

data_Alice['Period'] = data_Alice['Time'].diff()
periods_Alice = data_Alice['Period'].dropna()



data_Bob = pd.read_csv('Example Data/data_Bob_pendulum14m_25measurements.dat.txt', delim_whitespace=True, header=None)
data_Bob.columns = ['Period Number', 'Time']

data_Bob['Period'] = data_Bob['Time'].diff()
periods_Bob = data_Bob['Period'].dropna()

/var/folders/0f/55yfwn9x0kx5zvps17_ly6sw0000gn/T/ipykernel_90262/2512938491.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data_Alice = pd.read_csv('Example Data/data_Alice_pendulum14m_25measurements.dat.txt', delim_whitespace=True, header=None)
/var/folders/0f/55yfwn9x0kx5zvps17_ly6sw0000gn/T/ipykernel_90262/2512938491.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data_Bob = pd.read_csv('Example Data/data_Bob_pendulum14m_25measurements.dat.txt', delim_whitespace=True, header=None)


# Error Propagation for Pendulum

In [3]:
mu_L_Alice   =   14      #  meters
sig_L_Alice  =   0.5     #  meters
mu_T_Alice   =   periods_Alice.mean()
sig_T_Alice  =   periods_Alice.std()


mu_L_Bob   =   11      #    meters
sig_L_Bob  =   1.2     #    meters
mu_T_Bob   =   periods_Bob.mean()
sig_T_Bob  =   periods_Bob.std()

In [4]:
# Calculations etc.
g, L, T = symbols("g, L, T")
dg, dL, dT = symbols("sigma_g, sigma_L, sigma_T")

g = L*(2*pi/T)**2
lprint(latex(Eq(symbols('g'),g)))

dg = sqrt((g.diff(L) * dL)**2 + (g.diff(T) * dT)**2)
lprint(latex(Eq(symbols('sigma_g'), dg)))

fg = lambdify((L,T), g)
fdg = lambdify((L, dL, T, dT), dg)

vg_Alice = fg(mu_L_Alice, mu_T_Alice)
vdg_Alice = fdg(mu_L_Alice, sig_L_Alice, mu_T_Alice, sig_T_Alice)

vg_Bob = fg(mu_L_Bob, mu_T_Bob)
vdg_Bob = fdg(mu_L_Bob, sig_L_Bob, mu_T_Bob, sig_T_Bob)

lprint(fr'\mathrm{{Alice}}: g = ({vg_Alice:.2f} \pm {vdg_Alice:.2f})\, \mathrm{{m/s^2}}')
lprint(fr'\mathrm{{Bob}}: g = ({vg_Bob:.2f} \pm {vdg_Bob:.2f})\, \mathrm{{m/s^2}}')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Importing Example Data for Ball on Incline

In [5]:
data_ball1 = pd.read_csv('Example Data/data_NormDir_Medball1.txt', delim_whitespace=True, header=None)
data_ball1.columns = ['Time', 'Voltage']

data_ball2 = pd.read_csv('Example Data/data_RevDir_Medball1.txt', delim_whitespace=True, header=None)
data_ball2.columns = ['Period Number', 'Time']

data_ball2['Period'] = data_ball2['Time'].diff()
periods_ball2 = data_ball2['Period'].dropna()

/var/folders/0f/55yfwn9x0kx5zvps17_ly6sw0000gn/T/ipykernel_90262/1488354057.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data_ball1 = pd.read_csv('Example Data/data_NormDir_Medball1.txt', delim_whitespace=True, header=None)
/var/folders/0f/55yfwn9x0kx5zvps17_ly6sw0000gn/T/ipykernel_90262/1488354057.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data_ball2 = pd.read_csv('Example Data/data_RevDir_Medball1.txt', delim_whitespace=True, header=None)


# Error Propagation for Ball on Incline

In [6]:
#mu_theta   =  
#sig_theta  =  
#mu_D_ball  =  
#sig_D_ball = 
#mu_D_rail  =
#sig_D_rail =

#"a" is acceleration determined by s(t) = 1/2 at^2 + v0*t + s0

In [7]:
g, theta, D_ball, D_rail, a, a_norm, a_rev  = symbols("g, theta, D, d, a, a_norm, a_rev")
dg, dtheta, dD_ball, dD_rail, da = symbols("sigma_g, sigma_theta, sigma_D, sigma_d, sigma_a")
Delta_theta = Symbol(r'\Delta\theta')
dDelta_theta = Symbol(r'\sigma_{\Delta\theta}')

g_plus_Delta_Theta = a/sp.sin(theta + Delta_theta) * (1 + Rational(2,5) * D_ball**2/(D_ball**2 - D_rail**2))

dg_plus_Delta_Theta = sqrt((g_plus_Delta_Theta.diff(theta) * dtheta)**2 
                        + (g_plus_Delta_Theta.diff(D_ball) * dD_ball)**2 
                        + (g_plus_Delta_Theta.diff(D_rail) * dD_rail)**2 
                        + (g_plus_Delta_Theta.diff(a) * da)**2
                        + (g_plus_Delta_Theta.diff(Delta_theta) * dDelta_theta)**2)


g_minus_Delta_Theta = a/sp.sin(theta - Delta_theta) * (1 + Rational(2,5) * D_ball**2/(D_ball**2 - D_rail**2))

dg_minus_Delta_Theta = sqrt((g_minus_Delta_Theta.diff(theta) * dtheta)**2 
                        + (g_minus_Delta_Theta.diff(D_ball) * dD_ball)**2 
                        + (g_minus_Delta_Theta.diff(D_rail) * dD_rail)**2 
                        + (g_minus_Delta_Theta.diff(a) * da)**2
                        + (g_minus_Delta_Theta.diff(Delta_theta) * dDelta_theta)**2)


lprint(latex(Eq(symbols('g_+'), g_plus_Delta_Theta)))
lprint(latex(Eq(symbols('sigma_g_+'), dg_plus_Delta_Theta)))

lprint(latex(Eq(symbols('g_-'), g_minus_Delta_Theta)))
lprint(latex(Eq(symbols('sigma_g_-'), dg_minus_Delta_Theta)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [8]:
# Doing it with the definition of Delta theta:

g, theta, D_ball, D_rail, a, a_norm, a_rev  = symbols("g, theta, D, d, a, a_norm, a_rev")
dg, dtheta, dD_ball, dD_rail, da, da_norm, da_rev = symbols("sigma_g, sigma_theta, sigma_D, sigma_d, sigma_a, sigma_a_norm, sigma_a_rev")
Delta_theta = (a_norm - a_rev)*sin(theta)/((a_norm + a_rev)*cos(theta))
dDelta_theta = Symbol(r'\sigma_{\Delta\theta}')

g_plus_Delta_Theta_def = a/sp.sin(theta + Delta_theta) * (1 + Rational(2,5) * D_ball**2/(D_ball**2 - D_rail**2))

dg_plus_Delta_Theta_def = sqrt((g_plus_Delta_Theta_def.diff(theta) * dtheta)**2 
                        + (g_plus_Delta_Theta_def.diff(D_ball) * dD_ball)**2 
                        + (g_plus_Delta_Theta_def.diff(D_rail) * dD_rail)**2 
                        + (g_plus_Delta_Theta_def.diff(a) * da)**2
                        + (g_plus_Delta_Theta_def.diff(a_norm) * da_norm)**2
                        + (g_plus_Delta_Theta_def.diff(a_rev) * da_rev))


g_minus_Delta_Theta_def = a/sp.sin(theta - Delta_theta) * (1 + Rational(2,5) * D_ball**2/(D_ball**2 - D_rail**2))

dg_minus_Delta_Theta_def = sqrt((g_minus_Delta_Theta_def.diff(theta) * dtheta)**2 
                        + (g_minus_Delta_Theta_def.diff(D_ball) * dD_ball)**2 
                        + (g_minus_Delta_Theta_def.diff(D_rail) * dD_rail)**2 
                        + (g_minus_Delta_Theta_def.diff(a) * da)**2
                        + (g_minus_Delta_Theta_def.diff(a_norm) * da_norm)**2
                        + (g_minus_Delta_Theta_def.diff(a_rev) * da_rev))

In [9]:
#Defining functions
fg_plus_def = lambdify((theta, D_ball, D_rail, a_rev, a_norm, a), g_plus_Delta_Theta_def)
fdg_plus_def = lambdify((dtheta, dD_ball, dD_rail, da_rev, da_norm, da), dg_plus_Delta_Theta_def)

vg_plus_def = fg_plus_def()
vdg_plus_def = fdg_plus_def()

fg_minus_def = lambdify((theta, D_ball, D_rail, a_rev, a_norm, a), g_minus_Delta_Theta_def)
fdg_minus_def = lambdify((dtheta, dD_ball, dD_rail, da_rev, da_norm, da), dg_minus_Delta_Theta_def)

TypeError: _lambdifygenerated() missing 6 required positional arguments: 'theta', 'D', 'd', 'a_rev', 'a_norm', and 'a'